In [2]:
import pandas as pd
import geopandas as gpd

from dotenv import load_dotenv
import os

In [15]:
load_dotenv()  # Carga el archivo .env
# Ahora puedes acceder a las variables
ruta_unidad = os.getenv('RUTA_UNIDAD_LOCAL_DRIVE')
ruta_drive = os.getenv('RUTA_LOCAL_DRIVE')
print(ruta_unidad + ruta_drive)

C:\Documents\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON


In [18]:
path_grupos = ruta_unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'
path_catastro = ruta_unidad + r'\OneDrive - Ingenio Azucarero Guabira S.A\INFORMACION GENERAL\CATASTRO\NUEVO\catastro_S13_SABADO.shp'

output_path = ruta_unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DELTA_X'

In [21]:
def crear_xlsx_caneros():
    df_grupos = pd.read_excel(path_grupos, sheet_name='CODIGOS')
    # Seleccionar solo las columnas necesarias
    df_grupos = df_grupos[['CODIGO CAÑERO', 'NOMBRE CAÑERO', 'INSTITUCION']]
    # Renombrar las columnas
    df_grupos = df_grupos.rename(columns={
        'CODIGO CAÑERO': 'COD_CANERO',
        'NOMBRE CAÑERO': 'NOMBRE',
        'INSTITUCION': 'COD_INSTIT'
    })
    # Reordenar las columnas
    df_grupos = df_grupos[['COD_CANERO', 'COD_INSTIT', 'NOMBRE']]
    print(len(df_grupos), 'Cañeros registrados')
    archivo_salida = os.path.join(output_path, 'CANEROS.xlsx')
    df_grupos.to_excel(archivo_salida, index=False)
    print('Se exporto CANHEROS.xlsx exitosamente')

In [22]:
crear_xlsx_caneros()

1377 Cañeros registrados
Se exporto CANHEROS.xlsx exitosamente


In [23]:
def crear_xlsx_propiedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro[
        (gdf_catastro['obs'] == 'ok') &
        (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para PROPIEDAD.xlsx')
    df_pivot = gdf_cat_filtrado.groupby(['unidad_01', 'unidad_02'])['area'].sum().reset_index()
    df_pivot['unidad_01'] = df_pivot['unidad_01'].astype(int)
    df_repetidos = df_pivot[df_pivot.duplicated(subset='unidad_01', keep=False)]
    print('Cantidad de repetidos:', len(df_repetidos))
    df_pivot = df_pivot.rename(columns={
        'unidad_01': 'COD_PROPIE',
        'unidad_02': 'NOMBRE_PRO',
        'area': 'AREA_TOTAL'
    })
    # Definir nombre de archivo
    archivo_salida = os.path.join(output_path, 'PROPIEDAD.xlsx')
    # Exportar a Excel
    df_pivot.to_excel(archivo_salida, index=False)
    print('Se exporto PROPIEDAD.xlsx exitosamente')

In [24]:
crear_xlsx_propiedad()

Area Total de Catastro: 56139.12697181613 ha. para PROPIEDAD.xlsx
Cantidad de repetidos: 0
Se exporto PROPIEDAD.xlsx exitosamente


In [25]:
def crear_xlsx_caneho_propiedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro[
        (gdf_catastro['obs'] == 'ok') &
        (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para relacion CANERO-PROPIEDAD')
    # Asegurar que estamos trabajando sobre una copia, no una vista
    df_cat_filtrado = gdf_cat_filtrado[['unidad_03', 'unidad_01']].copy()
    # Convertir a entero
    df_cat_filtrado['unidad_03'] = df_cat_filtrado['unidad_03'].astype(int)
    df_cat_filtrado['unidad_01'] = df_cat_filtrado['unidad_01'].astype(int)
    df_grupos = pd.read_excel(path_grupos, sheet_name='CODIGOS')
    # Asegurarse de que ambas columnas estén en tipo entero
    df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
    # Realizar el merge (left join para mantener solo los registros del shapefile filtrado)
    df_combinado = df_cat_filtrado.merge(
        df_grupos,
        left_on='unidad_03',
        right_on='CODIGO CAÑERO',
        how='left'
    )
    # Eliminar los registros donde no se encontró coincidencia (por ejemplo, no tienen nombre cañero)
    df_combinado = df_combinado[df_combinado['NOMBRE CAÑERO'].notna()]
    # Seleccionar columnas específicas
    df_combinado = df_combinado[['unidad_01', 'GRUPO DE COSECHA']].copy()
    # Convertir las columnas a enteros
    df_combinado['unidad_01'] = df_combinado['unidad_01'].astype(int)
    df_combinado['GRUPO DE COSECHA'] = df_combinado['GRUPO DE COSECHA'].astype(int)
    # Crear columna 'idd' con el formato unidad_01_GRUPODECOSECHA
    df_combinado['idd'] = df_combinado['unidad_01'].astype(str) + '_' + df_combinado['GRUPO DE COSECHA'].astype(str)
    # Eliminar duplicados según la columna 'idd'
    df_combinado = df_combinado.drop_duplicates(subset='idd')
    # Crear una lista para guardar los resultados de cada grupo
    relaciones = []
    # Obtener todos los grupos únicos
    grupos = sorted(df_combinado['GRUPO DE COSECHA'].unique())
    for grupo in grupos:
        # Filtrar cañeros y propiedades del grupo actual
        df_cañeros = df_grupos[df_grupos['GRUPO DE COSECHA'] == grupo][['CODIGO CAÑERO']].drop_duplicates()
        df_props = df_combinado[df_combinado['GRUPO DE COSECHA'] == grupo][['unidad_01']].drop_duplicates()
        # Si hay cañeros y propiedades, hacer el producto cartesiano
        if not df_cañeros.empty and not df_props.empty:
            df_temp = df_cañeros.merge(df_props, how='cross')
            df_temp['GRUPO DE COSECHA'] = grupo
            relaciones.append(df_temp)
    # Unir todos los grupos en un solo DataFrame
    df_relacion_total = pd.concat(relaciones, ignore_index=True)
    # Eliminar la columna de grupo
    df_relacion_total = df_relacion_total.drop(columns=['GRUPO DE COSECHA'])
    # Renombrar las columnas
    df_relacion_total = df_relacion_total.rename(columns={
        'CODIGO CAÑERO': 'COD_CANERO',
        'unidad_01': 'COD_PROPIE'
    })
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'CANERO_PROPIEDAD.xlsx')
    # Exportar a Excel
    df_relacion_total.to_excel(archivo_salida, index=False)
    print('Se exporto CANERO_PROPIEDAD.xlsx exitosamente')

In [26]:
crear_xlsx_caneho_propiedad()

Area Total de Catastro: 56139.12697181613 ha. para relacion CANERO-PROPIEDAD
Se exporto CANERO_PROPIEDAD.xlsx exitosamente


In [27]:
gdf_catastro = gpd.read_file(path_catastro)
gdf_cat_filtrado = gdf_catastro[
    (gdf_catastro['obs'] == 'ok') &
    (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para relacion CANERO-PROPIEDAD')

Area Total de Catastro: 56139.12697181613 ha. para relacion CANERO-PROPIEDAD


In [29]:
# Crear DataFrame con la columna 'variedad', eliminando duplicados y nulos
df_variedad = gdf_cat_filtrado[['variedad']].dropna().drop_duplicates().reset_index(drop=True)

In [30]:
df_variedad

,variedad
0,UCG9020
1,RBB7726
2,RB2
3,CITTCA8522
4,RENOVACION
5,UCG9610
6,CITTCA0563
7,OTRAS
8,NA5626
9,SP835073


In [31]:
gdf_cat_filtrado[['unidad_01', 'unidad_03', 'unidad_05', 'area', 'variedad']]

,unidad_01,unidad_03,unidad_05,area,variedad
151,114.0,11860.0,L6.3.1,15.770016,UCG9020
204,771.0,11805.0,L6.5,3.630435,RBB7726
265,2286.0,100000.0,L1,3.345377,UCG9020
337,253.0,2488.0,L8.1,6.637463,RB2
425,639.0,2004.0,L1.1,4.561310,UCG9020
...,...,...,...,...,...
13451,541.0,4773.0,L7,3.931171,UCG9020
13455,2025.0,8023.0,NaN,1.653056,UCG9020
13456,986.0,13003.0,NaN,3.868736,UCG9020
13457,986.0,13003.0,NaN,3.427438,RBB7726


In [184]:
# Filtrar la columna 'variedad', eliminar nulos y duplicados
df_variedades = gdf_cat_filtrado[['variedad']].dropna().drop_duplicates()

# Exportar a Excel
archivo_variedades = os.path.join(output_path, 'VARIEDADES.xlsx')
df_variedades.to_excel(archivo_variedades, index=False)